In [51]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [52]:
'''
오류가 있다.
28일 뒤를 예측하도록 해야한다 
shift를 시켜야한다.
'''

'\n오류가 있다.\n28일 뒤를 예측하도록 해야한다 \nshift를 시켜야한다.\n'

In [53]:
df_train = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
calendar = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
price = pd.read_csv('m5-forecasting-accuracy/sell_prices.csv')
# df_test = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')

In [54]:
# # startpoints 찾아서 이전 데이터 지우기
# startpoints = np.zeros(df_train.shape[0])
# for idx in tqdm(range(df_train.shape[0])):
#     startpoints[idx]= np.where(df_train.iloc[idx,6:].values>0)[0].min().astype(int)

In [55]:
# start_dict = dict(zip(df_train['id'], startpoints))

In [56]:
# df_train['key'] = df_train['id'].apply(lambda x : x[:-11])


In [57]:
# test 데이터 변환
# df_test = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')

# lst = list(calendar[1941:]['d'])
# for i in range(1,29):
#     df_test = df_test.rename(columns = {'F%s'%i:lst[i-1]})

# # Unpivot
# df_test = pd.melt(df_test, id_vars=df_test.columns[:1], value_vars=df_test.columns[1:],
#        var_name = 'day', value_name = 'volume')

# df_test['key'] = df_test['id'].apply(lambda x : x[:-11])
# # df_train에 있는 기본 정보 붙이기
# df_test = pd.merge(df_test,df_train.loc[:,['key','item_id','dept_id','cat_id','store_id','state_id']],
#          on =['key'], how='left')

# df_train.drop(['key'],axis =1, inplace =True)
# df_test.drop(['key'], axis = 1, inplace =True)
    

In [58]:
# test_id = df_test['id']


### 전략
1. Unpivot만 해서 LGBM 하기 
2. 날짜 인코딩만해서 LGBM 사용하기
3. Data preprocessing 어떻게 할 것인지?
4. 최근 며칠간의 평균 판매액
5. rolling 추가
6. validation set을 6월 데이터 를 예측하는 것으로 해볼까?
  - 2015년 6월 데이터 예측 ? 

In [59]:
# F_1~28 만들기

tr_last = 1913
# tst = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')

# 1914~1941 까지 
for i in range(tr_last+1, tr_last+1+28):
    df_train['d_%s'%i] = 0


In [60]:
df_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
# Unpivot
df_train = pd.melt(df_train, id_vars=df_train.columns[:6], value_vars=df_train.columns[6:],
       var_name = 'day', value_name = 'volume')

    

In [62]:
# calendar 추가하기
df_train = pd.merge(df_train, calendar, left_on = 'day', right_on ='d')
# df_test = pd.merge(df_test, calendar, left_on = 'day', right_on ='d')

# df_train = df_train.fillna("NaN")
# df_test = df_test.fillna("NaN")

In [63]:
drop_cols = []
cat_cols = []
drop_cols += ['date','d','id']



In [64]:
# snap 합치기
snap = np.zeros(df_train.shape[0])
snap[df_train[(df_train['state_id']=='CA')&(df_train['snap_CA']==1)].index] +=1
snap[df_train[(df_train['state_id']=='TX')&(df_train['snap_TX']==1)].index] +=1
snap[df_train[(df_train['state_id']=='WI')&(df_train['snap_WI']==1)].index] +=1
df_train['snap'] = snap

# test
# snap = np.zeros(df_test.shape[0])
# snap[df_test[(df_test['state_id']=='CA')&(df_test['snap_CA']==1)].index] +=1
# snap[df_test[(df_test['state_id']=='TX')&(df_test['snap_TX']==1)].index] +=1
# snap[df_test[(df_test['state_id']=='WI')&(df_test['snap_WI']==1)].index] +=1
# df_test['snap'] = snap

drop_cols += ['snap_CA','snap_TX','snap_WI']



In [65]:

cat_cols += [ 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
             'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
            'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap']

In [66]:
# Sell price
df_train.head()
df_train = pd.merge(df_train, price)
# df_test = pd.merge(df_test,price)
# price

In [67]:
# df_train['d'] >='d_'+df_train['startpoint']

In [68]:
# # Start point 찾기 ::  40% 데이터를 날릴  수 있다.  -- price가 책정 된 날이 시작 점???

# df_train['startpoint'] = df_train['id'].map(start_dict).astype(int).astype(str)
# df_train['startpoints'] = df_train['d'] >='d_'+df_train['startpoint']
# print(df_train['startpoints'].value_counts())
# print(df_train.shape)
# # df_train = df_train[df_train['startpoints']]
# # print(df_train.shape)
# # df_train.drop(['startpoint','startpoints'],axis =1, inplace= True)

In [69]:
%%time

# 왜 Shift를 해야할까?
# Shift 28을 하지 않으면 예측값이 뒤로 가면갈 수록  F1->F28 예측 할 수 있는 변수가 줄어든게 된다.
# 28일은 한달을 의미한다. 최근 한달간의 경향성을 보는 것으로 보면 되겠다.
# 28일을 56일로 늘리면 안되나? - 최근 한달간의 경향성이 반영이 안되는 것이낙?
# https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50

df_train['volume_7'] = df_train[['id','volume']].groupby("id")['volume'].shift(7)
df_train['volume_28'] = df_train[['id','volume']].groupby("id")['volume'].shift(28)

print("mean")

df_train['rmean_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
df_train['rmean_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

df_train['rmean_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())
df_train['rmean_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).mean())


# df_train.dropna(inplace=True)


mean
Wall time: 3min 4s


In [70]:
cols =['event_name_1','event_type_1','event_name_2','event_type_2']
df_train[cols]= df_train[cols].fillna('NaN')
print(df_train.shape)
df_train.dropna(inplace =True)
# df_train.isnull().sum()
# print(df_train.shape)

(46881677, 32)


In [71]:
# %%time

# df_train['train'] = True
# # df_test['train']  = False
# # full_df = pd.concat([df_train,df_test])

# full_df['volume_shift_28'] =full_df[['id','volume']].groupby("id")['volume'].shift(28)

# print("mean")
# full_df['rmean_7'] = full_df[['id','volume_shift_28']].groupby("id")['volume_shift_28'].transform(lambda x: x.rolling(7 ,min_periods=1).mean())
# full_df['rmean_28'] = full_df[['id','volume_shift_28']].groupby("id")['volume_shift_28'].transform(lambda x: x.rolling(28 ,min_periods=1).mean())
# full_df['rmean_50'] = full_df[['id','volume_shift_28']].groupby("id")['volume_shift_28'].transform(lambda x: x.rolling(50 ,min_periods=1).mean())

# # print("std")
# # full_df['rstd_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7 ,min_periods=1).std())
# # full_df['rstd_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28 ,min_periods=1).std())
# # full_df['rstd_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(50 ,min_periods=1).std())

# # print("max")
# # full_df['rmax_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7 ,min_periods=1).max())
# # full_df['rmax_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28 ,min_periods=1).max())
# # full_df['rmax_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(50 ,min_periods=1).max())

# # print("min")
# # full_df['rmin_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7 ,min_periods=1).min())
# # full_df['rmin_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28 ,min_periods=1).min())
# # full_df['rmin_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(50 ,min_periods=1).min())

# # print("count")
# # full_df['rcnt_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
# # full_df['rcnt_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
# # full_df['rcnt_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)

# full_df = full_df.fillna(0)

# df_train = full_df[full_df['train'] ==True].drop(['train'], axis =1)
# df_test = full_df[full_df['train'] ==False].drop(['train'], axis =1)

# del full_df
# gc.collect()


45966979    HOBBIES_1_001_CA_1_validation
45966980    HOBBIES_1_001_CA_1_validation
45966981    HOBBIES_1_001_CA_1_validation
45966982    HOBBIES_1_001_CA_1_validation
45966983    HOBBIES_1_001_CA_1_validation
                        ...              
46881672      FOODS_3_825_WI_3_validation
46881673      FOODS_3_826_WI_3_validation
46881674      FOODS_3_826_WI_3_validation
46881675      FOODS_3_827_WI_3_validation
46881676      FOODS_3_827_WI_3_validation
Name: id, Length: 853680, dtype: object

In [76]:
df_train.head()
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
train_id = df_train['id']
df_test_id = df_train[df_train['day'].isin(testday)]['id']

In [135]:
df_train= df_train.drop(drop_cols,axis =1 )
# df_test =df_test.drop(drop_cols,axis =1)

In [136]:
df_train.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wm_yr_wk,weekday,wday,...,snap,sell_price,volume_7,volume_28,rmean_7_7,rmean_7_28,rmean_7_50,rmean_28_7,rmean_28_28,rmean_28_50
1010975,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_78,0,11112,Saturday,1,...,0.0,0.5,0.0,0.0,6.857143,1.785714,1.88,0.285714,1.285714,2.90
1010976,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_79,0,11112,Sunday,2,...,0.0,0.5,0.0,0.0,4.000000,1.714286,1.68,0.000000,1.142857,2.66
1010977,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_80,0,11112,Monday,3,...,0.0,0.5,8.0,0.0,5.142857,2.000000,1.76,0.000000,1.000000,2.36
1010978,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_81,5,11112,Tuesday,4,...,0.0,0.5,6.0,0.0,5.714286,2.214286,1.80,0.000000,0.928571,2.36
1010979,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_82,23,11112,Wednesday,5,...,0.0,0.5,6.0,0.0,4.714286,2.428571,1.88,0.000000,0.714286,2.36


In [137]:
# Encoding
for col in tqdm(cat_cols) :
    le = LabelEncoder()
#     le.fit()
#     le.fit(pd.concat([df_train[col],df_test[col]]))
    df_train[col] = le.fit_transform(df_train[col]).astype(np.int8)
#     df_test[col] = le.transform(df_test[col]).astype(np.int8)
    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [01:23<00:00,  5.56s/it]


In [138]:
# Test 분리하기
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
df_test = df_train[df_train['day'].isin(testday)]
df_train = df_train[~df_train['day'].isin(testday)]

# testday
df_train[df_train['day']== 'd_1914'].head()
# tst[tst['day']== 'd_1914']

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wm_yr_wk,weekday,wday,...,snap,sell_price,volume_7,volume_28,rmean_7_7,rmean_7_28,rmean_7_50,rmean_28_7,rmean_28_28,rmean_28_50


In [139]:
df_test.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wm_yr_wk,weekday,wday,...,snap,sell_price,volume_7,volume_28,rmean_7_7,rmean_7_28,rmean_7_50,rmean_28_7,rmean_28_28,rmean_28_50
45966979,-99,3,1,0,0,d_1914,0,6,1,2,...,0,8.38,1.0,1.0,0.857143,0.964286,0.96,1.142857,1.035714,1.08
45966980,-99,3,1,0,0,d_1915,0,6,5,3,...,0,8.38,1.0,0.0,1.000000,0.928571,0.98,0.857143,0.964286,1.04
45966981,-99,3,1,0,0,d_1916,0,6,6,4,...,0,8.38,1.0,0.0,1.142857,0.892857,0.96,0.571429,0.964286,1.00
45966982,-99,3,1,0,0,d_1917,0,6,4,5,...,0,8.38,3.0,0.0,1.428571,1.000000,1.02,0.571429,0.928571,1.00
45966983,-99,3,1,0,0,d_1918,0,6,0,6,...,0,8.38,0.0,0.0,1.285714,0.964286,1.00,0.428571,0.892857,1.00


In [140]:
# # Train_test_split  -- 가장 마지막 28일을 예측하는 것으로 짜보자
# # lst = [0 for x in range(df_train.shape[0])]

# x_train = df_train[~df_train['day'].isin(df_train['day'].unique()[-100:])]
# x_valid = df_train[df_train['day'].isin(df_train['day'].unique()[-100:])]

# y_train, y_valid = x_train['volume'], x_valid['volume']

# x_train = x_train.drop(['day','volume'], axis =1)
# x_valid = x_valid.drop(['day','volume'], axis =1)

In [141]:
%%time

from sklearn.model_selection import train_test_split

x_train , x_valid = train_test_split(df_train, test_size =0.15, random_state = 99)
y_train, y_valid = x_train['volume'], x_valid['volume']

x_train = x_train.drop(['day','volume'], axis =1)
x_valid = x_valid.drop(['day','volume'], axis =1)

Wall time: 34.4 s


In [142]:
x_valid.shape[0]/x_train.shape[0]

0.17647061833764963

In [143]:
from sklearn.metrics import mean_squared_log_error
seed = 99

In [144]:
# # lightbgm
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'metric': {'rmse'},#{'l2'},
#     #'subsample': 0.2,
#     'learning_rate': 0.1,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.9,
# #     'num_leaves': 128,
#     'alpha': 0.1,
#     'lambda': 0.1,
#     'n_jobs' :10 
# }

params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [145]:
%%time
# Modeling
lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=cat_cols)
lgb_eval = lgb.Dataset(x_valid, y_valid,categorical_feature=cat_cols)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000, #300,
                valid_sets=(lgb_train, lgb_eval),
#                     feval=rmsle,
#                 early_stopping_rounds= 50,#100,
                verbose_eval=50) #100)

C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[50]	training's rmse: 2.57197	valid_1's rmse: 2.57137
[100]	training's rmse: 2.49311	valid_1's rmse: 2.50216
[150]	training's rmse: 2.47031	valid_1's rmse: 2.48453
[200]	training's rmse: 2.45151	valid_1's rmse: 2.47037
[250]	training's rmse: 2.42994	valid_1's rmse: 2.45551
[300]	training's rmse: 2.41099	valid_1's rmse: 2.4442
[350]	training's rmse: 2.39564	valid_1's rmse: 2.43569
[400]	training's rmse: 2.38116	valid_1's rmse: 2.42732
[450]	training's rmse: 2.36879	valid_1's rmse: 2.42085
[500]	training's rmse: 2.35929	valid_1's rmse: 2.41633
[550]	training's rmse: 2.34934	valid_1's rmse: 2.41148
[600]	training's rmse: 2.34083	valid_1's rmse: 2.40751
[650]	training's rmse: 2.33338	valid_1's rmse: 2.40394
[700]	training's rmse: 2.32566	valid_1's rmse: 2.40106
[750]	training's rmse: 2.31947	valid_1's rmse: 2.39858
[800]	training's rmse: 2.31414	valid_1's rmse: 2.39638
[850]	training's rmse: 2.3075	valid_1's rmse: 2.3938
[900]	training's rmse: 2.3022	valid_1's rmse: 2.39187
[950]	training'

In [146]:
%%time
df_day = df_test['day']
df_test = df_test.drop(['day'],axis =1 )

predict = gbm.predict(df_test)
predict = np.around(predict)

Wall time: 9.71 s


In [147]:
df_test['id'] = df_test_id
df_test['volume'] = predict
df_test['day'] =df_day
tst =df_test.loc[:,['id','day','volume']]
tst['F']= [f"F{rank}" for rank in tst.groupby("id")["id"].cumcount()+1]
tst = tst.set_index(["id", "F" ]).unstack()["volume"].reset_index()
tst.sort_values("id", inplace = True)
tst.reset_index(drop=True, inplace = True)
tst =tst[['id']+["F%s"% x for x in range(1,29)]]

sub2 = tst.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([tst, sub2], axis=0, sort=False)

sub.fillna(0, inplace = True)
sub

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,2.0,2.0,2.0,2.0,4.0,2.0,1.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,FOODS_1_001_CA_2_validation,2.0,2.0,2.0,3.0,3.0,2.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,FOODS_1_001_CA_3_validation,3.0,2.0,2.0,3.0,3.0,2.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0
3,FOODS_1_001_CA_4_validation,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,FOODS_1_001_TX_1_validation,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,...,0.0,1.0,0.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30486,HOUSEHOLD_2_516_TX_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30487,HOUSEHOLD_2_516_WI_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
submission = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')
submission.shape

(60980, 29)

In [149]:
submission =pd.merge(submission['id'],sub, on = 'id')
submission.to_csv("submission_20200426_학습법 변경.csv",index=False)

In [ ]:
# te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
# te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
# te_sub.fillna(0., inplace = True)
# te_sub.sort_values("id", inplace = True)
# te_sub.reset_index(drop=True, inplace = True)
# te_sub.to_csv(f"submission_{icount}.csv",index=False)

In [ ]:
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

In [28]:
df_day = df_test['day']
df_test = df_test.drop(['day'],axis =1 )

In [29]:
%%time

predict = gbm.predict(df_test)
predict = np.around(predict)

Wall time: 19 s


In [30]:
submission = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')


In [33]:
df_test['id'] = test_id
df_test['predict']  = predict
df_test['day'] = df_day
df_test.head()

,item_id,dept_id,cat_id,store_id,state_id,volume,wm_yr_wk,weekday,wday,month,...,event_type_2,snap,sell_price,volume_shift_28,rmean_7,rmean_28,rmean_50,id,predict,day
0,-99,3,1,0,0,0,18,1,2,4,...,1,0,8.38,0.0,0.571429,0.464286,0.42,HOBBIES_1_001_CA_1_validation,1.0,d_1942
1,-99,3,1,0,0,0,18,1,2,4,...,1,0,8.38,0.0,0.000000,0.000000,0.00,HOBBIES_1_002_CA_1_validation,8.0,d_1942
2,-99,3,1,0,0,0,18,5,3,4,...,1,0,8.38,1.0,0.428571,0.464286,0.44,HOBBIES_1_003_CA_1_validation,1.0,d_1943
3,-99,3,1,0,0,0,18,5,3,4,...,1,0,8.38,0.0,0.000000,0.000000,0.00,HOBBIES_1_004_CA_1_validation,6.0,d_1943
4,-99,3,1,0,0,0,18,6,4,4,...,1,0,8.38,0.0,0.285714,0.464286,0.44,HOBBIES_1_005_CA_1_validation,1.0,d_1944


In [34]:
test2 = pd.pivot_table(df_test,index=['id'],columns = ['day'], values = 'predict').values

test2[test2<0] = 0


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
submission = pd.concat([submission.loc[:,'id'],pd.DataFrame(test2, columns = submission.columns[1:])],axis=1)
submission = round(submission.fillna(0))
submission.to_csv('submission_baseline3_rolling .csv',index=False)

In [36]:
submission


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,2.0,0.0,0.0,0.0,2.0,1.0,2.0,2.0,...,0.0,2.0,0.0,3.0,2.0,4.0,0.0,1.0,0.0,2.0
1,HOBBIES_1_002_CA_1_validation,0.0,2.0,0.0,0.0,0.0,6.0,1.0,2.0,7.0,...,0.0,6.0,1.0,1.0,5.0,3.0,0.0,1.0,1.0,0.0
2,HOBBIES_1_003_CA_1_validation,1.0,0.0,0.0,0.0,0.0,2.0,3.0,7.0,4.0,...,2.0,1.0,3.0,7.0,5.0,2.0,6.0,2.0,0.0,2.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,0.0,0.0,0.0,1.0,6.0,9.0,3.0,...,2.0,1.0,4.0,10.0,2.0,3.0,6.0,3.0,2.0,0.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,...,0.0,1.0,1.0,3.0,2.0,3.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.0,0.0,4.0,0.0,0.0,0.0,4.0,7.0,2.0,...,3.0,1.0,2.0,9.0,2.0,2.0,2.0,1.0,0.0,1.0
60976,FOODS_3_824_WI_3_evaluation,0.0,5.0,0.0,0.0,0.0,4.0,3.0,6.0,4.0,...,10.0,1.0,2.0,3.0,3.0,0.0,1.0,7.0,2.0,0.0
60977,FOODS_3_825_WI_3_evaluation,0.0,5.0,0.0,0.0,0.0,4.0,3.0,6.0,2.0,...,4.0,3.0,2.0,5.0,1.0,0.0,1.0,4.0,0.0,2.0
60978,FOODS_3_826_WI_3_evaluation,0.0,18.0,0.0,0.0,0.0,2.0,2.0,18.0,5.0,...,4.0,2.0,1.0,24.0,3.0,2.0,2.0,5.0,0.0,1.0


In [31]:
df_train.shape[0]-snap.sum()

28884852.0

In [32]:
snap = np.zeros(df_train.shape[0])

In [33]:
snap[[0,1,2]]=1
snap

array([1., 1., 1., ..., 0., 0., 0.])